# Plant Disease Detection

Dataset preparation

In [1]:
plnt_dses=r"D:\Data centr\IMG_data\Plant_Disease"

In [2]:
img_h=128
img_w=128
btch_sze=32

Preprocessing

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
data_gen=ImageDataGenerator(
    rescale=1./255, # Normalize pixel val
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

Split Training & Testing_dataset

In [6]:
train_data=data_gen.flow_from_directory(
    plnt_dses,
    target_size=(img_h,img_w),
    batch_size=btch_sze,
    class_mode="categorical",
    subset="training"
)

Found 70321 images belonging to 3 classes.


In [8]:
val_data=data_gen.flow_from_directory(
    plnt_dses,
    target_size=(img_h,img_w),
    batch_size=btch_sze,
    class_mode="categorical",
    subset="validation"
)

Found 17579 images belonging to 3 classes.


CNN Model preperation

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense, Dropout

In [21]:
model_CNN=Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3)), # 1st Convolutional Layer

    MaxPooling2D((2,2)), # Pooling Layer
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    
    Flatten(), # Flatten Feature Maps
    Dense(128,activation='relu'), # Fully Connected Layer
    Dropout(0.5), # Dropout for Regularization
    Dense(len(train_data.class_indices), activation='softmax') # Output Layer (multi-class)
])

c:\Users\himan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
model_CNN.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

<bound method Model.summary of <Sequential name=sequential_3, built=True>>

Train the model

In [29]:
epochs=20
history=model_CNN.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    steps_per_epoch=len(train_data),
    validation_steps=len(val_data)
    )

Epoch 1/20
1357/2198 ━━━━━━━━━━━━━━━━━━━━ 8:19 594ms/step - accuracy: 0.7995 - loss: 0.5214

KeyboardInterrupt: 

In [27]:
import matplotlib.pyplot as plt

In [28]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Training vs Validation Accuracy")
plt.show()

AttributeError: module 'matplotlib' has no attribute 'plot'

In [ ]:
val_loss,val_accuracy=model_CNN.evaluate(val_data)
print(f"Validation Loss: {val_loss*100:.2f}%")
print(f"Validation Accuracy: {val_accuracy*100:.2f}%")

In [ ]:
model_CNN.save("plant_disease.h5")
print(model_CNN.summary())